# Read all item IDs

In [1]:
import html

items_w_meta = set()
# skip_mag_sub = 0

with open('/home/shiv/Downloads/All_Amazon_Meta.json', 'r') as f: # 15,023,059 lines!
    line_cnt = 0
    for line in f:
        line_new = eval(line)
        
        items_w_meta.add(line_new['asin']) # collect all the item IDs
        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')  
print()
print("Total items", line_cnt)

# NOTES: lots of duplicate asins in meta file: only 14741571 items from 15023059 lines

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!
Total items 15023059


In [2]:
import pickle
items_pkl_path = f'/home/shiv/Documents/DataScience/Capstone/Data/items_w_meta.pkl'

with open(items_pkl_path, 'wb') as handle:
    pickle.dump(items_w_meta, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
import pickle
items_pkl_path = f'/home/shiv/Documents/DataScience/Capstone/Data/items_w_meta.pkl'
with open(items_pkl_path, 'rb') as handle:
    items_w_meta = pickle.load(handle)
print(len(items_w_meta))

14741571


# Read all reviews (157M + )

1. <s>save only reviews on or after the CUTOFF_DATE</s>
2. save only verified reviews
3. only for those products for which we have the meta data

In [3]:
import json
import pandas as pd

# CUTOFF_DATE = pd.to_datetime("2017-06-01")

ratings_items = []
line_cnt = 0
with open('/home/shiv/Documents/DataScience/Capstone/Data/All_Amazon_Review_5.json', 'r') as f:
    for line in f:
        review = json.loads(line) # overall, verified, reviewerID, asin, unixReviewTime, reviewText, summary
#         ts = pd.to_datetime(review['unixReviewTime'], unit='s')
#         if ts >= CUTOFF_DATE and review['verified'] and review['asin'] in items_w_meta:
        if review['verified'] and review['asin'] in items_w_meta:
            ratings_items.append([review['reviewerID'], review['asin'],
                                  review['overall'], review['unixReviewTime']])
      
        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')
print()
print(line_cnt) # 157260920
print(len(ratings_items))

# NOTES: even though it is curated, there aren't 5 reviews per product nor 5 reviews per user
# So, choosing to only look at verified reviews and for products that have the meta information.
# Down to 133M+ reviews

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!

In [5]:
ratings_df = pd.DataFrame(ratings_items, columns = ['reviewerID', 'asin', 
                                                    'rating', 'unixReviewTime'])
ratings_df.head()

,reviewerID,asin,rating,unixReviewTime
0,AEUZVK0XRMI40,B000HA3CEY,5.0,1363824000
1,A1U4R8DP7MLOYP,B000HA3CEY,2.0,1354060800
2,A1JIXKFDTEC6DZ,B000HA3CEY,5.0,1225411200
3,A11AP8Z9IF3NH2,B001EJZNXK,5.0,1361836800
4,A2YJWVLJ21CUFG,B001EJZNXK,4.0,1357171200


In [7]:
ratings_csv_path = f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews.csv'
ratings_df.to_csv(ratings_csv_path, header=False, index=False)

# Restart the kernel

In [1]:
import pandas as pd
ratings_csv_path = f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_5.csv'
ratings_df = pd.read_csv(ratings_csv_path, header=None)
ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixReviewTime']

In [2]:
def get_min_reviews_prod_per_user(ratings_df, ret_min_prod, ret_min_user):
    prod_df = ratings_df[['reviewerID','asin']].groupby(['asin']).count()
    min_prod = -1
    if ret_min_prod:
        min_prod = prod_df['reviewerID'].min()
        print("min reviews per item", min_prod) #, prod_df['reviewerID'].max())
    user_df = ratings_df[['reviewerID','asin']].groupby(['reviewerID']).count()
    min_user = -1
    if ret_min_user:
        min_user = user_df['asin'].min()
        print("min reviews per user", min_user) #, user_df['asin'].max())
    return min_prod, prod_df, min_user, user_df

min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, True, True)

min reviews per item 5
min reviews per user 5


# Remove items with less than MIN_REVIEWS reviews

This is a multi-round process. In each round, we do the following:

1. Filter the ratings dataframe by removing products that have less than MIN_REVIEWS reviews
2. Next, check if the remaining users have MIN_REVIEWS_PER_USER reviews
3. If both MIN_REVIEWS and MIN_REVIEWS_PER_USER conditions are satisfied, we are done!
4. Otherwise filter the ratings dataframe by removing users that have less than MIN_REVIEWS_PER_USER reviews
5. Next, check if the remaining items have MIN_REVIEWS reviews
6. If both MIN_REVIEWS and MIN_REVIEWS_PER_USER conditions are satisfied, we are done!

In [3]:
MIN_REVIEWS = 20
MIN_REVIEWS_PER_USER = 20

print("# users", ratings_df['reviewerID'].nunique(), "#items", ratings_df['asin'].nunique())

# users 8534835 #items 3214887


In [4]:
round = 1
while True:
    print("Round #", round)
    
    prod_list = list(prod_df[prod_df['reviewerID'] >= MIN_REVIEWS].index)
    print("# items:", len(prod_list))
    
    ratings_df = ratings_df[ratings_df['asin'].isin(prod_list)]
    print(ratings_df.shape)
    
    min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, False, True)
    min_prod = MIN_REVIEWS
    
    if min_prod >= MIN_REVIEWS and min_user >= MIN_REVIEWS_PER_USER:
        break    

    user_list = list(user_df[user_df['asin'] >= MIN_REVIEWS_PER_USER].index)
    print("# users", len(user_list))
    
    ratings_df = ratings_df[ratings_df['reviewerID'].isin(user_list)]
    print(ratings_df.shape)
    
    min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, True, False)
    min_user = MIN_REVIEWS_PER_USER
    
    if min_prod >= MIN_REVIEWS and min_user >= MIN_REVIEWS_PER_USER:
        break    
    round += 1

Round # 1
# items: 1070664
(108501158, 4)
min reviews per user 1
# users 1284496
(50858773, 4)
min reviews per item 1
Round # 2
# items: 552846
(44457426, 4)
min reviews per user 1
# users 1040451
(40210494, 4)
min reviews per item 2
Round # 3
# items: 497273
(39222851, 4)
min reviews per user 3
# users 999330
(38455791, 4)
min reviews per item 5
Round # 4
# items: 486864
(38261068, 4)
min reviews per user 14
# users 990998
(38103482, 4)
min reviews per item 15
Round # 5
# items: 484648
(38061565, 4)
min reviews per user 17
# users 989235
(38028150, 4)
min reviews per item 16
Round # 6
# items: 484197
(38019593, 4)
min reviews per user 18
# users 988826
(38011832, 4)
min reviews per item 16
Round # 7
# items: 484088
(38009765, 4)
min reviews per user 19
# users 988743
(38008188, 4)
min reviews per item 19
Round # 8
# items: 484066
(38007770, 4)
min reviews per user 19
# users 988722
(38007371, 4)
min reviews per item 19
Round # 9
# items: 484062
(38007295, 4)
min reviews per user 19
# 

In [5]:
ratings_df.to_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_{MIN_REVIEWS}.csv', 
                  header=False, index=False)

# Get the meta information for the products in all_reviews_{MIN_REVIEWS}

In [6]:
# import pandas as pd
# ratings_df = pd.read_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_{MIN_REVIEWS}.csv',
#                          header=None)
# ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixReviewTime']
print(ratings_df.shape)
ratings_df

(38007219, 4)


,reviewerID,asin,rating,unixReviewTime
5,A14E6RS86U3IWJ,B001EJY2IC,5.0,1363737600
6,A279VRO0YVM43V,B001EJY2IC,4.0,1362355200
9,A1HJ4BHRNFBGSX,B001EJY2IC,5.0,1473033600
11,A139FFBSZTFUAT,B001EJY2IC,5.0,1468281600
12,A2MBPLWN8MKY93,B001EJY2IC,5.0,1466208000
...,...,...,...,...
128135433,A1WAJWOFBE4PPR,B01HB9Q7CW,5.0,1537747200
128135642,A1MCN1E5GNFNXJ,B01HI9W5HQ,1.0,1538006400
128135643,A3IAV917ZH50SE,B01HI9W5HQ,4.0,1535068800
128135644,A2RPMPBKX1MWR7,B01HI9W5HQ,5.0,1534550400


In [7]:
ratings_df.reset_index(inplace=True,drop=True)

set_items = set(ratings_df['asin'].unique()) # all unique items in the ratings dataframe
print("# users", ratings_df['reviewerID'].nunique(), "# items", len(set_items))

# users 988718 # items 484062


## Clean up meta information

Meta information consists of the following:

1. main category
2. categories
3. price
4. title

- Each of the columns can come with HTML escape sequences that need to be removed.
- Each of the columns can be empty!
- price and title columns are very noisy!
- There are some items that have too many categories. Only the first 5 are saved to conserve space

In [8]:
import html

items = []
meta_items = set()

with open('/home/shiv/Downloads/All_Amazon_Meta.json', 'r') as f: # 15,023,059 lines!
    line_cnt = 0
    empty_main_cat = 0
    empty_lst_cat = 0
    empty_price = 0
    empty_title = 0
    missing_price_cnt = 0
    missing_title_cnt = 0
    dups_cnt = 0
    
    for line in f:
        line_new = eval(line)

        asin = line_new['asin']
        if asin in set_items:
            lst_vals = []
            # asin
            lst_vals.append(line_new['asin'])

            # main_cat
            main_cat = line_new['main_cat']
            main_cat = html.unescape(main_cat)
            lst_vals.append(main_cat)
            if line_new['main_cat'] == '':
                empty_main_cat += 1
            
            # categories
            lst_categories = line_new['category']
            if len(lst_categories) > 5:
                lst_categories = lst_categories[:5]
            cats = []
            for cat in lst_categories:
                cats.append(html.unescape(cat))
            if len(cats) == 0:
                empty_lst_cat += 1
            lst_vals.append('|'.join(cats))
            
            # price
            if 'price' in line_new:
                lst_vals.append(line_new['price'])
                if line_new['price'].strip() == '':
                    empty_price += 1
            else:
                missing_price_cnt += 1
                lst_vals.append('')

            # title    
            if 'title' in line_new:
                title = line_new['title'].strip()
                title = html.unescape(title)
                if title == '':
                    empty_title += 1
                    title = 'Unknown'
                lst_vals.append(title)                
            else:
                missing_title_cnt += 1
                lst_vals.append('')

            if asin in meta_items:
                lst_vals.append(1)
                dups_cnt += 1
            else:
                lst_vals.append(0)
                meta_items.add(asin)
                
            items.append(lst_vals)

        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')        

print()
print(len(items), "duplicates", dups_cnt)
print("main category", empty_main_cat)
print("lst category", empty_lst_cat)
print("title", missing_title_cnt, empty_title)
print("price", missing_price_cnt, empty_price)

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!
498489 duplicates 14427
main category 1269
lst category 19596
title 0 104
price 0 129512


In [9]:
# Assert if we did not get the meta data for all unique items
lst_no_meta_items = list(set_items - meta_items)
assert(len(lst_no_meta_items) == 0)
# ratings_df = ratings_df[~ratings_df['asin'].isin(lst_no_meta_items)]

In [10]:
items_df = pd.DataFrame(items, columns = ['asin', 'main_cat', 'category', 'price', 'title', 'duplicate'])
# sorted(items_df['main_cat'].unique())

## Fix html tags appearing in main_cat, use alt instead

In [12]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.found_tag = False
        self.recorded_alt = False
        self.alt_cat = ''
        
    def handle_starttag(self, tag, attrs):
        # print("Encountered a start tag:", attrs)
        if tag == 'img':
            self.found_tag = True
            for a in attrs:
                if a[0] == 'alt':
                    self.recorded_alt = True
                    self.alt_cat = a[1].title()

    def handle_endtag(self, tag):
        # print("Encountered an end tag :", tag)
        self.found_tag = False

    def handle_data(self, data):
        # print("Encountered some data  :", data)
        pass

main_cat_dict = {}
parser = MyHTMLParser()
lst_main_cat = items_df.main_cat.unique()

for cat in lst_main_cat:
    parser.recorded_alt = False
    parser.feed(cat)
    if parser.recorded_alt:
        # print(parser.alt_cat)
        main_cat_dict[cat] = parser.alt_cat
    else:
        main_cat_dict[cat] = cat

In [13]:
items_df['main_cat_tr'] = items_df['main_cat'].apply(lambda x: main_cat_dict[x])
# sorted(items_df['main_cat_tr'].unique())

## Pick first category if main_cat_tr is empty

In [14]:
items_df['main_cat_tr'] = items_df[['main_cat_tr','category']].apply(lambda x: x['category'].split('|')[0] if x['main_cat_tr'] == '' else x['main_cat_tr'], axis=1)

## Consolidate main category

- Some of the main categories have very few items. We fold them into an overarching category by looking at the categories list (if not empty) and/or the title.

In [15]:
print(len(sorted(items_df['main_cat_tr'].unique())))
items_df[['main_cat_tr', 'main_cat']].groupby(['main_cat_tr']).count()

51


,main_cat
main_cat_tr,
All Beauty,1513
All Electronics,10417
Amazon Devices,668
Amazon Fashion,55759
Amazon Fire TV,5
Amazon Home,68701
Apple Products,43
Appliances,249
"Arts, Crafts & Sewing",7018


In [20]:
main_cat_map = { # Transform the main_cat if the count for a main category is low
    'MEMBERSHIPS & SUBSCRIPTIONS': 'Home & Kitchen',
    'Entertainment': 'Collectibles',
    'Amazon Fire TV': 'Amazon Devices',
    'Fire Phone' : 'Amazon Devices',
    'Sports Collectibles': 'Collectibles',
    'Collectibles & Fine Art' : 'Collectibles',
    'Collectible Coins': 'Collectibles',
    'Buy a Kindle': 'Kindle',
    'Kindle Store': 'Kindle'
}

In [21]:
items_df['main_cat_tr'] = items_df['main_cat_tr'].apply(lambda x: main_cat_map[x] if x in main_cat_map else x)

In [22]:
len(sorted(items_df['main_cat_tr'].unique()))

45

## Fix category lists:
1. remove less frequently occuring categories
2. remove main_category from this list of (sub)categories

In [23]:
from collections import defaultdict
from tqdm import tqdm

cat_count = defaultdict(lambda: 0)
with tqdm(total=items_df.shape[0]) as pbar:
    categories = items_df['category'].values
    for row in categories:
        for cat in row.split('|'):
            cat_count[cat] += 1
        pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████████████| 498489/498489 [00:00<00:00, 1533226.08it/s]


In [24]:
items_df['category_tr'] = ''
def update_category(row):
    categories = row['category']
    new_cat = []
    # print(categories)
    for cat in categories.split('|'):
        if cat_count[cat] >= 5 and cat != row['main_cat_tr']:
            new_cat.append(cat)
    if len(new_cat) > 0:
        row['category_tr'] = '|'.join(new_cat)
    else:
        row['category_tr'] = ''
    return row
items_df = items_df.apply(update_category, axis=1)

In [25]:
# Ensure that the duplicate rows have no new information
tmp_df = items_df[items_df['duplicate']==1]
with tqdm(total=tmp_df.shape[0]) as pbar:
    for _, row in tmp_df.iterrows():
        dup_rows = items_df[items_df['asin']==row['asin']]
        assert(dup_rows.shape[0]==2) # only 2 rows share an asin
        assert(dup_rows.iloc[0]['category'] == dup_rows.iloc[1]['category'])
        assert(dup_rows.iloc[0]['main_cat_tr'] == dup_rows.iloc[1]['main_cat_tr'])
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 14427/14427 [08:23<00:00, 28.67it/s]


## Drop the unnecessary columns; save all_meta_{MIN_REVIEWS}

In [26]:
items_df.drop(columns=['main_cat','category','duplicate'], inplace=True)
print(items_df.shape)
items_df.drop_duplicates(inplace=True)
print(items_df.shape)
items_df.head()

(498489, 5)
(484062, 5)


,asin,price,title,main_cat_tr,category_tr
0,7709260373,$23.99 - $29.99,LJYH Children's Collar Motorcycle Faux Leather...,Amazon Fashion,"Clothing, Shoes & Jewelry|Boys|Clothing|Jacket..."
1,9792252916,,"Casio Men's Quartz Resin Casual Watch, Color:B...",Amazon Fashion,"Clothing, Shoes & Jewelry|Men|Watches|Wrist Wa..."
2,B00001QHXX,,Politically Incorrect Nixon Vinyl Mask,Amazon Fashion,"Clothing, Shoes & Jewelry|Costumes & Accessori..."
3,B00001TOXD,$4.99,Adult Witch Broom,Toys & Games,"Clothing, Shoes & Jewelry|Costumes & Accessori..."
4,B00001T38Y,$24.98,Star Wars Stormtrooper Deluxe Adult Mask,Amazon Fashion,"Clothing, Shoes & Jewelry|Costumes & Accessori..."


In [27]:
items_df.rename(columns={'main_cat_tr': 'main_cat', 'category_tr': 'category'}, inplace=True)

In [28]:
items_df.to_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_meta_{MIN_REVIEWS}.csv', 
                header=False, index=False)

# Prepare for wide_deep

1. Restart the kernel
2. Read the ratings and items dataframes
3. Prepare Electronics, Home and Games datasets

In [1]:
import pandas as pd
MIN_REVIEWS = 20

DATA_DIR = '/home/shiv/Documents/DataScience/Capstone/Data/wide_deep/'
ratings_df = pd.read_csv(DATA_DIR + f'all_reviews_{MIN_REVIEWS}.csv', header=None)
ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixTimeStamp']     

In [2]:
items_df = pd.read_csv(DATA_DIR + f'all_meta_{MIN_REVIEWS}.csv', header=None)
# items_df.head()

In [3]:
items_df.columns=['asin','price','title','main_cat','category']
items_df['category'].fillna('', inplace=True)
items_df['price'].fillna('$$$', inplace=True)

In [4]:
# Sample to show how to drop a main category

# mag_subs = items_df[items_df['main_cat']=='Magazine Subscriptions']['asin'].values
# ratings_df=ratings_df[~ratings_df['asin'].isin(mag_subs)]
# items_df=items_df[~items_df['asin'].isin(mag_subs)]
# ratings_df.reset_index(inplace=True,drop=True)
# items_df.reset_index(inplace=True,drop=True)

In [5]:
len(sorted(items_df['main_cat'].unique()))

45

## Use all the main categories; if only specific main categories are needed, skip to [Electronics](#electronics)

In [6]:
from tqdm import tqdm
import pickle

# Step 3: convert reviewerID to userID (0 based)
print("Converting reviewerID to userID")
reviewers_dict = {}
reviewer_id = 0
with tqdm(total=ratings_df.shape[0]) as pbar:
    for _, row in ratings_df.iterrows():
        if row['reviewerID'] not in reviewers_dict:
            reviewers_dict[row['reviewerID']] = reviewer_id
            reviewer_id += 1
        pbar.update(1)

ratings_df['userID'] = ratings_df['reviewerID'].apply(lambda x: reviewers_dict[x])

# Step 4: save the reviewerID dict (perhaps for UI)
users_pkl_path = DATA_DIR + f'users_all_{MIN_REVIEWS}.pkl'

with open(users_pkl_path, 'wb') as handle:
    pickle.dump(reviewers_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

del reviewers_dict
ratings_df.drop(columns=['reviewerID'], inplace=True) # we will henceforth use userID

# Step 5: left merge the ratings with items on asin
data = ratings_df.merge(items_df, on=['asin'], how='left')
print("merged dataframe", data.shape)
data['category'] = data['category'].astype('string')

del ratings_df

# Step 6: convert asin to itemID (0 based)
print("Converting asin to itemID")
items_dict = {}
item_id = 0

with tqdm(total=items_df.shape[0]) as pbar:
    for _, row in items_df.iterrows():
        if row['asin'] not in items_dict:
            items_dict[row['asin']] = item_id
            item_id += 1
        pbar.update(1)
data['itemID'] = data['asin'].apply(lambda x: items_dict[x])

# Step 7: save the itemID dict (for UI)
items_pkl_path = DATA_DIR + f'items_all_{MIN_REVIEWS}.pkl'

with open(items_pkl_path, 'wb') as handle:
    pickle.dump(items_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

del items_dict
data.drop(columns=['asin'], inplace=True) # we will henceforth use itemID

tqdm.pandas()
lst_main_cat = items_df.main_cat.unique()

# Step 8: add genre; 
# currently, consider the main categories or if an item in category matches the main category list
data['genre'] = ''

def update_category(row):
    categories = row['category']
    new_cat = []
    new_cat.append(row['main_cat'])
    # print(categories)
    for cat in categories.split('|'):
        if cat.strip() != '' and (cat in lst_main_cat):
            new_cat.append(cat)
    row['genre'] = '|'.join(new_cat)
    return row
data = data.progress_apply(update_category, axis=1)

# Step 9: save the prepared dataset to be used by wide_deep model
data.drop(columns=['category'], inplace=True)
data.sort_values('unixTimeStamp', inplace=True)
data.reset_index(inplace=True, drop=True)
data = data[['userID','itemID', 'rating','genre','unixTimeStamp','title','price','main_cat','category']]
data.to_csv(DATA_DIR + f'wide_deep_amzn_all_{MIN_REVIEWS}.csv', 
             header=False, index=False)

Converting reviewerID to userID


100%|███████████████████████████████████████████████████████████████████████████████████| 38007219/38007219 [07:52<00:00, 80368.65it/s]


merged dataframe (38007219, 8)
Converting asin to itemID


100%|███████████████████████████████████████████████████████████████████████████████████████| 484062/484062 [00:07<00:00, 62080.04it/s]


<a id="electronics"></a>
# Prepare Electronics, Home, Games datasets

Steps:

1. Filter the items dataframe to remove all items whose main_cat is not in the list of main_categories for each umbrella category (e.g. Electronics)
2. Filter the rating dataframe to remove all reviews that do not belong to the list of item IDs left in the items dataframe after step 1.
3. Convert hexadecimal reviewerID in the ratings dataframe to userID (0 based)
4. Save the reviewerID dict in a pkl file
5. Left merge ratings and items filtered dataframe on asin (hexadecimal)
6. Convert hexadecimal asin to itemID (0 based)
7. Save the itemID dict (for UI) in a pkl file
8. Add genre; currently, consider the main categories or if an item in category matches the sub category list
9. Save the prepared dataset for use by modeling

In [21]:
import pickle
DATA_DIR = '/home/shiv/Documents/DataScience/Capstone/Data/wide_deep/'

def prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, dataset):
    data_dir = DATA_DIR + dataset + '/'
    
    # Step 1: filter the items dataframe
    items_filtered_df = items_df[items_df['main_cat'].isin(lst_main_cat)].copy()
    items_filtered_ids = items_filtered_df['asin'].values

    # Step 2: filter the ratings dataframe
    ratings_filtered_df = ratings_df[ratings_df['asin'].isin(items_filtered_ids)].copy()
    
    print(dataset)
    print("Num ratings:", ratings_filtered_df.shape[0])
    print("users", ratings_filtered_df['reviewerID'].nunique(), "items", ratings_filtered_df['asin'].nunique())
    
    # Step 3: convert reviewerID to userID (0 based)
    reviewers_dict = {}
    reviewer_id = 0
    for _, row in ratings_filtered_df.iterrows():
        if row['reviewerID'] not in reviewers_dict:
            reviewers_dict[row['reviewerID']] = reviewer_id
            reviewer_id += 1
            
    ratings_filtered_df['userID'] = ratings_filtered_df['reviewerID'].apply(lambda x: reviewers_dict[x])

    # Step 4: save the reviewerID dict (perhaps for UI)
    users_pkl_path = data_dir + f'users_{dataset[0].lower()}_{MIN_REVIEWS}.pkl'

    with open(users_pkl_path, 'wb') as handle:
        pickle.dump(reviewers_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    ratings_filtered_df.drop(columns=['reviewerID'], inplace=True) # we will henceforth use userID

    # Step 5: left merge the ratings with items on asin
    data = ratings_filtered_df.merge(items_filtered_df, on=['asin'], how='left')
    print("merged dataframe", data.shape)
    data['category'] = data['category'].astype('string')
    
    # Step 6: convert asin to itemID (0 based)
    items_dict = {}
    item_id = 0

    for _, row in data.iterrows():
        if row['asin'] not in items_dict:
            items_dict[row['asin']] = item_id
            item_id += 1
    data['itemID'] = data['asin'].apply(lambda x: items_dict[x])
    
    # Step 7: save the itemID dict (for UI)
    items_pkl_path = data_dir + f'items_{dataset[0].lower()}_{MIN_REVIEWS}.pkl'

    with open(items_pkl_path, 'wb') as handle:
        pickle.dump(items_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        
    data.drop(columns=['asin'], inplace=True) # we will henceforth use itemID
    
    # Step 8: add genre; 
    # currently, consider the main categories or if an item in category matches the sub category list
    data['genre'] = ''

    def update_category(row):
        categories = row['category']
        new_cat = []
        new_cat.append(row['main_cat'])
        # print(categories)
        for cat in categories.split('|'):
            if cat.strip() != '' and ((cat in lst_main_cat) or (cat in lst_sub_cat)):
                new_cat.append(cat)
        row['genre'] = '|'.join(new_cat)
        return row
    data = data.apply(update_category, axis=1)
    
    # Step 9: save the prepared dataset to be used by wide_deep model
    # data.drop(columns=['category'], inplace=True)
    data.reset_index(inplace=True, drop=True)
    data = data[['userID','itemID', 'rating','genre','unixTimeStamp','title','price','main_cat','category']]
    data.to_csv(data_dir + f'wide_deep_amzn_{dataset[0].lower()}_{MIN_REVIEWS}.csv', 
                 header=False, index=False)
    
    del items_filtered_df
    del ratings_filtered_df
    del data

### Electronics

In [22]:
# main_cat = 'Electronics'
lst_main_cat = ['All Electronics', 'Amazon Devices', 'Apple Products', 
                 'Camera & Photo', 'Car Electronics', 'Cell Phones & Accessories', 'Computers',
                 'Electronics', 'GPS & Navigation', 'Home Audio & Theater', 'Industrial & Scientific',
                 'Portable Audio & Accessories']

lst_sub_cat = ['Accessories','Computers & Accessories','Office Products','Video Games',
               'Accessories & Supplies','Tools & Home Improvement','Computer Accessories & Peripherals',
               'Audio & Video Accessories', 'Automotive', 'Office & School Supplies',
               'Car & Vehicle Electronics', 'Industrial & Scientific','Sports & Outdoors','Office Electronics',
               'Home & Kitchen','Musical Instruments','Portable Audio & Video','Electrical',
               'Clothing, Shoes & Jewelry','Toys & Games','Laptop Accessories','Home Audio',
               'Controllers','Computer Components','Sports & Fitness']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Electronics")

Electronics
Num ratings: 5613183
users 830668 items 63725
merged dataframe (5613183, 8)


### Home

In [24]:
# main_cat = 'Home'
lst_main_cat = ['Amazon Home', 'Appliances', 'Home & Kitchen', 
                'Patio, Lawn & Garden', 'Tools & Home Improvement']

lst_sub_cat = ['Kitchen & Dining', 'Industrial & Scientific', 'Power & Hand Tools',
               'Automotive', 'Arts, Crafts & Sewing', 'Office Products',
               'Electronics', 'Sports & Outdoors', 'Home Dcor', 'Accessories', 'Hand Tools',
               'Office & School Supplies', 'Gardening & Lawn Care',
               'Hardware', 'Storage & Organization', 'Lighting & Ceiling Fans',
               'Kitchen Utensils & Gadgets', 'Electrical', 'Furniture', 'Pet Supplies',
               'Building Supplies', 'Bedding', 'Sports & Fitness', 'Safety & Security',
               'Outdoor Recreation', 'Power Tool Parts & Accessories', 'Kitchen & Bath Fixtures',
               'Parts & Accessories', 'Small Appliances', 'Replacement Parts',
               'Crafting', 'Sewing', 'Tools & Equipment', 'Outdoor Dcor', 'Patio Furniture & Accessories',
               'Grills & Outdoor Cooking', 'Power Tools', 'Rough Plumbing', 'Bath',
               'Bakeware', 'Accessories & Supplies', 'Heating, Cooling & Air Quality',
               'Outdoor Power Tools', 'Outdoor Lighting', 'Paint, Wall Treatments & Supplies',
               'Home Dcor Accents', 'Pools, Hot Tubs & Supplies', 'Bathroom Fixtures',
               'Test, Measure & Inspect', 'Bathroom Accessories', 'Personal Protective Equipment',
               'Fasteners', 'Vacuums & Floor Care', 'Clothing & Closet Storage', 'Exterior Accessories',
               'Replacement Parts & Accessories', 'Desk Accessories & Workspace Organizers',
               'Outdoor Cooking Tools & Accessories', 'HVAC']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Home")

Home
Num ratings: 7575829
users 882963 items 92562
merged dataframe (7575829, 8)


### Games

In [29]:
# main_cat = 'Games'
lst_main_cat = ['Sports & Outdoors', 'Toys & Games', 'Video Games']
lst_sub_cat = ['Sports & Fitness', 'Accessories', 'Clothing, Shoes & Jewelry', 'Home & Kitchen',
               'Outdoor Recreation', 'Clothing', 'Electronics', 'Games', 
               'Sports & Outdoor Play', 'Cycling', 'Exercise & Fitness', 'Camping & Hiking',
               'Leisure Sports & Game Room', 'Hunting & Fishing', 'Retro Gaming & Microconsoles',
               'Costumes & Accessories', 'Dress Up & Pretend Play', 'Shoes', 'Golf', 'Hobbies',
               'Water Sports', 'Controllers', 'Xbox One', 'PlayStation 3', 'Motorcycle & Powersports',
               'Arts & Crafts', 'Replacement Parts', 'Xbox 360', 'Other Sports', 'Crafting',
               'Wii', 'Learning & Education', 'Sports', 'Active', 'Consoles',
               'Painting, Drawing & Art Supplies', 'Audio & Video Accessories', 'Nintendo 3DS & 2DS',
               'Athletic', 'Skates, Skateboards & Scooters', 'Building Toys', 'Building Sets']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Games")

Games
Num ratings: 3735507
users 748773 items 56219
merged dataframe (3735507, 8)


# Code to discover relevant sub categories to build the lst_sub_cat

In [ ]:
from collections import defaultdict

ratings_g_df = pd.read_csv(DATA_DIR + 'wide_deep_amzn_g_20.csv', header=None, low_memory=False)
ratings_g_df.columns=['userID','itemID', 'rating','genre','unixTimeStamp','title','price','main_cat','category']
ratings_g_df['category'].fillna('', inplace=True)
ratings_g_df['price'].fillna('$$$', inplace=True)
    
lst = list(ratings_g_df['category'].unique())
word_count = defaultdict(int)
for l in lst:
    for w in l.split('|'):
        word_count[w] += 1
dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

# Additional datasets: Books, Health & Personal Care, Fashion

In [ ]:
# main_cat = 'Books'
lst_main_cat = ['Books', 'Kindle', 'Audible audiobooks']

In [ ]:
# main_cat = 'Health & Personal Care'
lst_main_cat = ['All Beauty', 'Health & Personal Care', 'Luxury Beauty']

# main_cat = 'Fashion'
lst_main_cat = ['Amazon Fashion', 'Clothing, Shoes & Jewelry']

# Prepare for SLiRec

1. Restart the kernel
2. Read the ratings and items dataframes
3. Prepare Electronics, Home and Games datasets

In [1]:
import pandas as pd
import os
import _pickle as cPickle
import random

MIN_REVIEWS = 20

DATA_DIR = '/home/shiv/Documents/DataScience/Capstone/Data/wide_deep/'
ratings_df = pd.read_csv(DATA_DIR + f'all_reviews_{MIN_REVIEWS}.csv', header=None)
ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixTimeStamp']      

In [2]:
items_df = pd.read_csv(DATA_DIR + f'all_meta_{MIN_REVIEWS}.csv', header=None)
items_df.columns=['asin','price','title','main_cat','category']
items_df['category'].fillna('', inplace=True)
items_df['price'].fillna('$$$', inplace=True)

In [3]:
len(sorted(items_df['main_cat'].unique()))

45

In [4]:
DATA_DIR = '/home/shiv/Documents/DataScience/Capstone/Data/slirec/'
TEST_NUM_NGS = 49
VALID_NUM_NGS = 4

In [5]:
# Helper functions

def write_reviews(data_dir, ratings_filtered_df):
    reviews_writefile = data_dir + '/reviews.output'
    reviews_w = open(reviews_writefile, 'w')
    for _, row in ratings_filtered_df.iterrows():
        reviews_w.write(
            row["reviewerID"]
            + "\t"
            + row["asin"]
            + "\t"
            + str(row["unixTimeStamp"])
            + "\n"
        )
    reviews_w.close()
    return reviews_writefile

def write_meta(data_dir, items_df):
    meta_writefile = data_dir + "/meta.output"
    meta_w = open(meta_writefile, "w")

    for _, row in items_df.iterrows():
        meta_w.write(row["asin"] + "\t" + row["main_cat"] + "\n")
    meta_w.close()
    return meta_writefile

def write_instance_output(reviews_file, meta_file):
    # For every user, create a list of items reviews sorted by ascending timestamp
    print("start create instances...")
    dirs, _ = os.path.split(reviews_file)
    output_file = os.path.join(dirs, "instance.output")

    f_reviews = open(reviews_file, "r")
    user_dict = {}
    item_list = []
    
    for line in f_reviews:
        line = line.strip()
        reviews_things = line.split("\t") # user_id asin unix_ts
        if reviews_things[0] not in user_dict:
            user_dict[reviews_things[0]] = []
        user_dict[reviews_things[0]].append((line, float(reviews_things[-1]))) # note append whole line, ts
        item_list.append(reviews_things[1])

    f_meta = open(meta_file, "r")
    meta_dict = {}
    for line in f_meta:
        line = line.strip()
        meta_things = line.split("\t") # asin category
        if meta_things[0] not in meta_dict:
            meta_dict[meta_things[0]] = meta_things[1]

    f_output = open(output_file, "w")
    num_default_cat = 0
    for user_id in user_dict:
        sorted_user_behavior = sorted(user_dict[user_id], key=lambda x: x[1]) # x[1]: tuple (line, ts)
        for line, _ in sorted_user_behavior:
            user_things = line.split("\t") # user_id, asin, ts
            asin = user_things[1]
            if asin in meta_dict:
                f_output.write("1" + "\t" + line + "\t" + meta_dict[asin] + "\n") # positive
            else:
                num_default_cat += 1
                f_output.write("1" + "\t" + line + "\t" + "default_cat" + "\n") # positive

    f_reviews.close()
    f_meta.close()
    f_output.close()
    print("Num default categories:", num_default_cat)
    assert(num_default_cat == 0)
    return output_file

def write_preprocessed_output(sampled_instance_file):
    print("start data processing...")
    dirs, _ = os.path.split(sampled_instance_file)
    output_file = os.path.join(dirs, "preprocessed.output")

    f_input = open(sampled_instance_file, "r")
    f_output = open(output_file, "w")
    user_count = {}

    for line in f_input:
        line = line.strip()
        user = line.split("\t")[1] # ["label", "user_id", "item_id", "timestamp", "cate_id"]
        if user not in user_count:
            user_count[user] = 0
        user_count[user] += 1

    f_input.seek(0)
    i = 0
    last_user = None
    for line in f_input:
        line = line.strip()
        user = line.split("\t")[1]
        if user == last_user:
            if i < user_count[user] - 2:
                f_output.write("train" + "\t" + line + "\n")
            elif i < user_count[user] - 1:
                f_output.write("valid" + "\t" + line + "\n")
            else:
                f_output.write("test" + "\t" + line + "\n")
        else:
            last_user = user
            i = 0
            if i < user_count[user] - 2:
                f_output.write("train" + "\t" + line + "\n")
            elif i < user_count[user] - 1:
                f_output.write("valid" + "\t" + line + "\n")
            else:
                f_output.write("test" + "\t" + line + "\n")
        i += 1
    return output_file

def write_train_valid_test_data(data_dir, output_file):
    train_file = os.path.join(data_dir, r'train_data')
    valid_file = os.path.join(data_dir, r'valid_data')
    test_file = os.path.join(data_dir, r'test_data')
    
    f_input = open(output_file, "r") # preprocessed.output
    f_train = open(train_file, "w")
    f_valid = open(valid_file, "w")
    f_test = open(test_file, "w")
    min_sequence = 1

    print("train, valid, test positive data generating...")
    last_user_id = None
    for line in f_input:
        line_split = line.strip().split("\t")
        tfile = line_split[0]        # train/valid/test
        label = int(line_split[1])   # label 1
        user_id = line_split[2]      # user
        item_id = line_split[3]      # asin
        date_time = line_split[4]    # ts
        category = line_split[5]     # category

        if tfile == "train":
            fo = f_train
        elif tfile == "valid":
            fo = f_valid
        elif tfile == "test":
            fo = f_test
        if user_id != last_user_id:
            item_id_list = [] # collect all items
            cate_list = []
            dt_list = []
        else:
            history_clk_num = len(item_id_list)
            cat_str = ""
            iid_str = ""
            dt_str = ""
            for c1 in cate_list:
                cat_str += c1 + ","
            for iid in item_id_list:
                iid_str += iid + ","
            for dt_time in dt_list:
                dt_str += dt_time + ","
            if len(cat_str) > 0:
                cat_str = cat_str[:-1]
            if len(iid_str) > 0:
                iid_str = iid_str[:-1]
            if len(dt_str) > 0:
                dt_str = dt_str[:-1]
            if history_clk_num >= min_sequence:
                fo.write(
                    line_split[1]
                    + "\t"
                    + user_id
                    + "\t"
                    + item_id
                    + "\t"
                    + category
                    + "\t"
                    + date_time
                    + "\t"
                    + iid_str
                    + "\t"
                    + cat_str
                    + "\t"
                    + dt_str
                    + "\n"
                )
        last_user_id = user_id
        if label:
            item_id_list.append(item_id)
            cate_list.append(category)
            dt_list.append(date_time)
    return train_file, valid_file, test_file

In [6]:
def gen_vocab_pkl_files(data_dir, train_file):
    user_vocab = os.path.join(data_dir, r'user_vocab.pkl')
    item_vocab = os.path.join(data_dir, r'item_vocab.pkl')
    cate_vocab = os.path.join(data_dir, r'category_vocab.pkl')

    f_train = open(train_file, "r") # NOTE: only train_file

    user_dict = {}
    item_dict = {}
    cat_dict = {}

    print("vocab generating...")
    for line in f_train:
        arr = line.strip("\n").split("\t") # label uid asin category ts asin_list, cat_list, ts_list
        uid = arr[1]
        iid = arr[2]
        cat = arr[3]
        iid_list = arr[5]
        cat_list = arr[6]

        if uid not in user_dict:
            user_dict[uid] = 0
        user_dict[uid] += 1
        if iid not in item_dict:
            item_dict[iid] = 0
        item_dict[iid] += 1
        if cat not in cat_dict:
            cat_dict[cat] = 0
        cat_dict[cat] += 1
        if len(iid_list) == 0:
            print("No history", uid)
            continue
        for m in iid_list.split(","):
            if m not in item_dict:
                item_dict[m] = 0
            item_dict[m] += 1
        for c in cat_list.split(","):
            if c not in cat_dict:
                cat_dict[c] = 0
            cat_dict[c] += 1

    sorted_user_dict = sorted(user_dict.items(), key=lambda x: x[1], reverse=True) # sorted on popularity
    sorted_item_dict = sorted(item_dict.items(), key=lambda x: x[1], reverse=True)
    sorted_cat_dict = sorted(cat_dict.items(), key=lambda x: x[1], reverse=True)

    uid_voc = {}
    index = 0
    for key, value in sorted_user_dict:
        uid_voc[key] = index
        index += 1

    iid_voc = {}
    iid_voc["default_mid"] = 0
    index = 1
    for key, value in sorted_item_dict:
        iid_voc[key] = index
        index += 1

    cat_voc = {}
    cat_voc["default_cat"] = 0
    index = 1
    for key, value in sorted_cat_dict:
        cat_voc[key] = index
        index += 1

    cPickle.dump(uid_voc, open(user_vocab, "wb"))
    cPickle.dump(iid_voc, open(item_vocab, "wb"))
    cPickle.dump(cat_voc, open(cate_vocab, "wb"))    

In [7]:
def gen_neg_samples(sampled_instance_file, item2cate, valid_file, test_file):
    columns = ["label", "user_id", "item_id", "timestamp", "cate_id"]
    ns_df = pd.read_csv(sampled_instance_file, sep="\t", names=columns)
    items_with_popular = list(ns_df["item_id"])

    # valid negative sampling
    print("start valid negative sampling")
    with open(valid_file, "r") as f:
        valid_lines = f.readlines()
    write_valid = open(valid_file, "w")
    for line in valid_lines:
        write_valid.write(line)
        words = line.strip().split("\t")
        positive_item = words[2]
        count = 0
        neg_items = set()
        while count < VALID_NUM_NGS:
            neg_item = random.choice(items_with_popular)
            if neg_item == positive_item or neg_item in neg_items:
                continue
            count += 1
            neg_items.add(neg_item)
            words[0] = "0"
            words[2] = neg_item
            words[3] = item2cate[neg_item]
            write_valid.write("\t".join(words) + "\n")

    # test negative sampling
    print("start test negative sampling")
    with open(test_file, "r") as f:
        test_lines = f.readlines()
    write_test = open(test_file, "w")
    for line in test_lines:
        write_test.write(line)
        words = line.strip().split("\t")
        positive_item = words[2]
        count = 0
        neg_items = set()
        while count < TEST_NUM_NGS:
            neg_item = random.choice(items_with_popular)
            if neg_item == positive_item or neg_item in neg_items:
                continue
            count += 1
            neg_items.add(neg_item)
            words[0] = "0"
            words[2] = neg_item
            words[3] = item2cate[neg_item]
            write_test.write("\t".join(words) + "\n")    

In [8]:
def prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, dataset):
    data_dir = DATA_DIR + dataset + '/'
    
    # Step 1: filter the items dataframe
    items_filtered_df = items_df[items_df['main_cat'].isin(lst_main_cat)].copy()
    items_filtered_ids = items_filtered_df['asin'].values

    # Step 2: filter the ratings dataframe
    ratings_filtered_df = ratings_df[ratings_df['asin'].isin(items_filtered_ids)].copy()
    
    print(dataset)
    print("Num ratings:", ratings_filtered_df.shape[0])
    print("users", ratings_filtered_df['reviewerID'].nunique(), "items", ratings_filtered_df['asin'].nunique())
    
    # Step 3: write the reviews output file (reviews preprocessing)
    # reviews_writefile = data_dir + 'reviews.output'
    reviews_writefile = write_reviews(data_dir, ratings_filtered_df)
    
    # Step 4: write the meta output file (meta preprocessing)
    # meta_writefile = data_dir + 'meta.output'
    meta_writefile = write_meta(data_dir, items_df)
    
    # Step 5: create the instance.output
    # instance_output = data_dir + 'instance.output'
    instance_output = write_instance_output(reviews_writefile, meta_writefile)
    
    # Step 6: create item2category dictionary
    instance_df = pd.read_csv(
        instance_output,
        sep="\t",
        names=["label", "user_id", "item_id", "timestamp", "cate_id"],
    )
    
    # tmp_df = instance_df[["item_id","cate_id"]].drop_duplicates()
    # print(tmp_df.shape)
    item2cate = instance_df.set_index("item_id")["cate_id"].to_dict() # item_id/asin: category
    print(instance_df.shape, instance_df['item_id'].nunique(), len(item2cate))
    del instance_df
    
    # everything in the instance.output is either train, valid or test
    sampled_instance_file = instance_output
    
    # Step 7: split into train (all records for a user except the last 2), 
    # validation (second last), test (last)
    # output_file = data_dir + 'preprocessed.output'
    output_file = write_preprocessed_output(sampled_instance_file)
    train_file, valid_file, test_file = write_train_valid_test_data(data_dir, output_file)

    # Step 8: generate vocabulary files        
    gen_vocab_pkl_files(data_dir, train_file)
    
    # Step 9: negative sampling (validation, test only); train is done inline
    gen_neg_samples(sampled_instance_file, item2cate, valid_file, test_file)

## Electronics

In [9]:
# main_cat = 'Electronics'
lst_main_cat = ['All Electronics', 'Amazon Devices', 'Apple Products', 
                 'Camera & Photo', 'Car Electronics', 'Cell Phones & Accessories', 'Computers',
                 'Electronics', 'GPS & Navigation', 'Home Audio & Theater', 'Industrial & Scientific',
                 'Portable Audio & Accessories']

lst_sub_cat = ['Accessories','Computers & Accessories','Office Products','Video Games',
               'Accessories & Supplies','Tools & Home Improvement','Computer Accessories & Peripherals',
               'Audio & Video Accessories', 'Automotive', 'Office & School Supplies',
               'Car & Vehicle Electronics', 'Industrial & Scientific','Sports & Outdoors','Office Electronics',
               'Home & Kitchen','Musical Instruments','Portable Audio & Video','Electrical',
               'Clothing, Shoes & Jewelry','Toys & Games','Laptop Accessories','Home Audio',
               'Controllers','Computer Components','Sports & Fitness']

prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, 'Electronics')

Electronics
Num ratings: 5613183
users 830668 items 63725
start create instances...
Num default categories: 0
(5613183, 5) 63725 63725
start data processing...
train, valid, test positive data generating...
vocab generating...
start valid negative sampling
start test negative sampling
